In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用AutoML自然语言和Vertex AI进行情感分析

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/sentiment_analysis/Sentiment_Analysis.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/sentiment_analysis/Sentiment_Analysis.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/sentiment_analysis/Sentiment_Analysis.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

## 目录
* [概述](#section-1)
* [目标](#section-2)
* [数据集](#section-3)
* [成本](#section-4)
* [加载数据](#section-5)
* [准备训练数据](#section-6)
* [在 Vertex AI 中创建数据集](#section-7)
* [使用 Vertex AI 训练模型](#section-8)
* [部署模型到终端](#section-9)
* [预测](#section-10)
* [查看可视化结果](#section-11)
* [清理](#section-12)

## 概览
<a name="section-1"></a>

本笔记本演示了如何使用AutoML自然语言在斯坦福电影评论数据集上执行情感分析，并如何部署情感分析模型到Vertex AI以获取预测结果。

*注意：此笔记本文件是为在[Vertex AI工作台托管笔记本](https://console.cloud.google.com/vertex-ai/workbench/list/managed)实例上使用Python（本地）内核而开发的。此笔记本的一些组件可能无法在其他笔记本环境中运行。*

了解更多关于[Vertex AI工作台](https://cloud.google.com/vertex-ai/docs/workbench/introduction)和[文本数据的情感分析](https://cloud.google.com/vertex-ai/docs/training-overview#sentiment_analysis_for_text)。

### 目标
在本教程中，您将学习如何训练和部署一个AutoML情感分析模型，并进行预测。

本教程使用以下谷歌云ML服务：

- `Vertex AI数据集`
- `Vertex AI模型`
- `AutoML训练`
- `Vertex AI预测`

执行的步骤包括：

- 加载所需数据。
- 预处理数据。
- 为模型选择所需的数据。
- 将数据集加载到Vertex AI托管数据集中。
- 使用AutoML文本训练训练情感模型。
- 评估模型。
- 在Vertex AI上部署模型。
- 获取预测结果。
- 进行清理操作。

### 数据集
<a name="section-3"></a>

这个笔记本中使用的数据集是[斯坦福情感树库数据集](https://nlp.stanford.edu/sentiment/)的一部分，其中包含来自电影评论的短语和它们相应的情感分数。

### 成本
<a name="section-4"></a>

本教程使用Google Cloud的以下可计费组件：

- Vertex AI
- Cloud Storage

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage的定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

In [ ]:
! pip3 install --upgrade --quiet wordcloud 

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 fsspec \
                                 gcsfs 

### 仅限Colab使用：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您会获得$300的免费信用额度用于计算/存储成本。

2. [确保为您的项目启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，请安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 设置区域

**可选**：更新'REGION'变量以指定您想使用的区域。了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行实时教程会话，您可能正在使用共享的测试账户或项目。为了避免在创建的资源上发生命名冲突，您为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经通过身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 请查看如何向您的服务账号授予云存储权限，网址为https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才会创建：运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
import os
from typing import List, Optional, Union

import matplotlib.pyplot as plt
import pandas as pd
from google.cloud import aiplatform, storage
from wordcloud import STOPWORDS, WordCloud

# 加载数据
<a name="section-5"></a>

从云存储源加载数据集的短语和分数。

In [ ]:
phrases = pd.read_csv(
    "gs://cloud-samples-data/vertex-ai/managed_notebooks/sentiment_analysis/stanford_sentiment_treebank/sentiment_phrases.txt",
    sep="|",
)
phrases.columns = ["text", "phrase ids"]
scores = pd.read_csv(
    "gs://cloud-samples-data/vertex-ai/managed_notebooks/sentiment_analysis/stanford_sentiment_treebank/sentiment_labels.txt",
    sep="|",
)
df = phrases.merge(scores, how="left", on="phrase ids")
print(df.head(5))

In [ ]:
print(max(df["sentiment values"]), min(df["sentiment values"]))

数据本身不包含任何特征名称，因此需要重新命名其列。数据框**短语**包含所有短语及其ID，用`|`分隔，而**分数**包含所有短语ID和相应的情感评分，用`|`分隔。

### 创建标签

通过将标签分数映射到以下范围，将标签分为四个类别： 

[0， 0.25]， (0.25， 0.5]， (0.5， 0.75]， (0.75， 1.0]

In [ ]:
# Define the class labels
VERYNEGATIVE = 0
NEGATIVE = 1
POSITIVE = 2
VERYPOSITIVE = 3

In [ ]:
bins = [0, 0.25, 0.5, 0.75, 1]
labels = [VERYNEGATIVE, NEGATIVE, POSITIVE, VERYPOSITIVE]
df["label"] = pd.cut(df["sentiment values"], bins=bins, labels=labels)
print(df.head())

## 准备训练数据
<a name="section-6"></a>

要使用AutoML自然语言训练情感分析模型，您需要提供带有表示情感变化的值标记的要训练内容的样本。

情感标签被编码为整数，范围从0（相对负面）到您选择的最大值（正面）。例如，如果您想确定情感是负面的、正面的还是中性的，您可以使用情感分数为0（负面）、1（中性）和2（正面）来标记训练数据。如果您想要捕捉更多细微差别的情感等级，则可以将最负面的情感标记为0，并将最正面的情感标记为4。在这种情况下，数据集的最大情感分数（sentiment_max）将为4。

对于训练，您选择一个由极端正面和负面样本组成的原始数据子集。在这里，情感的最大值为1。在**ml_use**列中，您提供实例属于TRAIN、VALIDATION或TEST样本，或让Vertex AI随机分配。

CSV文件中的每一行或实例都指代单个文档。以下示例显示了有效CSV文件的一般格式：

**[ml_use]**,**gcs_file_uri**|**"inline_text"**,**sentiment**,**sentimentMax**

有关更多信息，请访问官方文档 [documentation](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text#sentiment-analysis) 有关准备情感分析数据的详细信息。

选择数据集的子集

In [ ]:
subset_data = df[df["label"].isin([VERYNEGATIVE, VERYPOSITIVE])].reset_index(drop=True)
subset_data.head()

In [ ]:
subset_data["label"] = subset_data["label"].apply(lambda x: 1 if x == 3 else 0)
subset_data["ml_use"] = ""
subset_data["sentimentMax"] = 1
subset_data = subset_data[["ml_use", "text", "label", "sentimentMax"]]
print(subset_data.head())

### 创建一个csv

In [ ]:
FILE_NAME = "sentiment_data.csv"
subset_data.to_csv(FILE_NAME, index=False)
# Upload the saved model file to Cloud Storage
BLOB_PATH = "sentiment_analysis/"
BLOB_NAME = os.path.join(BLOB_PATH, FILE_NAME)
bucket = storage.Client().bucket(BUCKET_URI[5:])
blob = bucket.blob(BLOB_NAME)
blob.upload_from_filename(FILE_NAME)

## 在 Vertex AI 中创建数据集
<a name="section-7"></a>

以下代码使用 Python 中的 Vertex AI SDK 来创建数据集并导入数据。

In [ ]:
def import_data_text_sentiment_analysis(
    project: str,
    location: str,
    display_name: str,
    src_uris: Union[str, List[str]],
    sync: bool = True,
):
    aiplatform.init(project=project, location=location)

    ds = aiplatform.TextDataset.create(
        display_name=display_name,
        gcs_source=src_uris,
        import_schema_uri=aiplatform.schema.dataset.ioformat.text.sentiment,
        sync=sync,
    )

    print(ds.display_name)
    print(ds.resource_name)
    return ds

为受管理的数据集设置显示名称。

In [ ]:
DATASET_NAME = "[your-dataset-display-name]"  # @param {type:"string"}

In [ ]:
if DATASET_NAME == "" or DATASET_NAME == "[your-dataset-display-name]":
    DATASET_NAME = "sentimentanalysis"

# Create a Vertex AI managed dataset resource
src_uris = [f"{BUCKET_URI}/sentiment_analysis/sentiment_data.csv"]
dataset = import_data_text_sentiment_analysis(
    PROJECT_ID, REGION, DATASET_NAME, src_uris
)

## 使用顶点AI训练模型
<a name="section-8"></a>

以下代码使用顶点AI的Python SDK在上面创建的数据集上训练模型。您可以从控制台顶点AI的数据集部分或上面创建的数据集对象的资源名称中获取数据集ID。您可以通过设置fraction_split变量来指定训练数据在训练、验证和测试集之间的拆分方式。

**注意**: 模型训练作业可能会花费超过120分钟才能完成。

In [ ]:
def create_training_pipeline_text_sentiment_analysis(
    project: str,
    location: str,
    display_name: str,
    dataset_id: str,
    model_display_name: Optional[str] = None,
    sentiment_max: int = 10,
    training_fraction_split: float = 0.8,
    validation_fraction_split: float = 0.1,
    test_fraction_split: float = 0.1,
    sync: bool = True,
):
    aiplatform.init(project=project, location=location)

    job = aiplatform.AutoMLTextTrainingJob(
        display_name=display_name,
        prediction_type="sentiment",
        sentiment_max=sentiment_max,
    )

    text_dataset = aiplatform.TextDataset(dataset_id)

    model = job.run(
        dataset=text_dataset,
        model_display_name=model_display_name,
        training_fraction_split=training_fraction_split,
        validation_fraction_split=validation_fraction_split,
        test_fraction_split=test_fraction_split,
        sync=sync,
    )

    print(model.display_name)
    print(model.resource_name)
    print(model.uri)
    return model

为AutoML训练作业设置显示名称。

In [ ]:
JOB_NAME = "[your-training-job-display-name]"  # @param {type:"string"}

In [ ]:
if JOB_NAME == "" or JOB_NAME == "[your-training-job-display-name]":
    JOB_NAME = "sentimentanalysis_job"

# Using the created Text dataset, create and run the training job
dataset_id = dataset.resource_name.split("/")[-1]
print(dataset_id)
model = create_training_pipeline_text_sentiment_analysis(
    PROJECT_ID, REGION, JOB_NAME, dataset_id, sentiment_max=1
)

将模型部署到端点
<a name="section-9"></a>

创建端点
创建一个Vertex AI端点来部署模型。

In [ ]:
def create_endpoint(
    project: str,
    display_name: str,
    location: str,
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint

为终端设置显示名称。

In [ ]:
ENDPOINT_NAME = "[your-endpoint-name]"  # @param {type:"string"}

In [ ]:
if ENDPOINT_NAME == "[your-endpoint-name]":
    ENDPOINT_NAME = "sentiment-analysis-endpoint"

# Create Endpoint
endpoint = create_endpoint(PROJECT_ID, ENDPOINT_NAME, REGION)

### 部署模型

以下代码使用了 Python 的 Vertex AI SDK 将模型部署到一个端点。

In [ ]:
model.deploy(endpoint=endpoint)

print(model.display_name)
print(model.resource_name)

## 预测
<a name="section-10"></a>

将模型部署到端点后，使用Vertex AI SDK请求在线预测。筛选出训练过程中未使用的数据，并选择较长的评论来测试模型。

In [ ]:
def predict_text_sentiment_analysis_sample(endpoint, content):
    print(content)
    response = endpoint.predict(instances=[{"content": content}], parameters={})

    for prediction_ in response.predictions:
        print(prediction_)

In [ ]:
test_data_pos = df[df["label"].isin([POSITIVE])].reset_index(drop=True)
test_data_neg = df[df["label"].isin([NEGATIVE])].reset_index(drop=True)

test_data_neg = test_data_neg.text.values[200:]
random.shuffle(test_data_neg)

In [ ]:
test_data_pos = test_data_pos.text.values[200:]
random.shuffle(test_data_pos)

这里是对正面样本的预测结果。模型在预测正面评论的情感方面做得很好。第一条和最后一条评论的预测是假阴性。

In [ ]:
for review in test_data_pos[0:10]:
    predict_text_sentiment_analysis_sample(endpoint, review)

这是负面评论的预测结果。在10条评论中，有7条是负面评论，这些负面评论被正确识别为负面情感。

In [ ]:
for review in test_data_neg[0:10]:
    predict_text_sentiment_analysis_sample(endpoint, review)

## 评论可视化
<a name="section-11"></a>

将数据中的正面和负面评论可视化。

In [ ]:
data_pos = df[df["label"].isin([VERYPOSITIVE])].reset_index(drop=True)
data_neg = df[df["label"].isin([VERYNEGATIVE])].reset_index(drop=True)

data_neg = data_neg.text.values

In [ ]:
data_pos = data_pos.text.values

通过删除常见词汇来创建词云，突出代表积极和消极样本的词汇。

In [ ]:
# Python program to generate WordCloud
def plot_word_cloud(data, common_words):
    comment_words = ""
    stopwords = set(STOPWORDS)
    for val in data:
        tokens = val.split()
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()
            for each in common_words:
                if each in tokens[i]:
                    tokens[i] = ""
                    break

        comment_words += " ".join(tokens) + " "

    wordcloud = WordCloud(
        width=800,
        height=800,
        background_color="white",
        stopwords=stopwords,
        min_font_size=10,
    ).generate(comment_words)

    plt.figure(figsize=(8, 8), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)

    plt.show()

绘制负面评论的词云。

In [ ]:
plot_word_cloud(
    data_neg,
    [
        "movie",
        "film",
        "story",
        "audience",
        "director",
        "watch",
        "seem",
        "world",
        "one",
        "make",
        "way",
        "character",
        "much",
        "time",
        "even",
        "take",
        "s",
        "n't",
        "will",
        "may",
        "re",
        "plot",
        "good",
        "comedy",
        "made",
    ],
)

制作一个正面点评的词云。

In [ ]:
plot_word_cloud(
    data_pos,
    [
        "movie",
        "film",
        "story",
        "audience",
        "director",
        "watch",
        "seem",
        "world",
        "one",
        "make",
        "way",
        "character",
        "much",
        "time",
        "even",
        "take",
        "s",
        "n't",
        "will",
        "may",
        "re",
        "plot",
        "made",
    ],
)

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Undeploy the model from the endpoint
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete the dataset
dataset.delete()

将 `delete_bucket` 设置为 **True** 以删除云存储桶。

In [ ]:
delete_bucket = False
# Delete the Cloud storge bucket
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI